In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [3]:
links = ["https://web.archive.org/web/20190524134324/https://socialblade.com/youtube/channel/UCqJNTGwKmgMCwfYDbnz514Q/monthly",
        "https://web.archive.org/web/20190524171902/https://socialblade.com/youtube/user/mmoshaya/monthly",
        "https://web.archive.org/web/20190525094206/https://socialblade.com/youtube/user/banderitax/monthly",
        "https://web.archive.org/web/20190524075036/https://socialblade.com/youtube/user/arabgamesnetwork/monthly",
        "https://web.archive.org/web/20190830014628/https://socialblade.com/youtube/channel/UCBzrJ1zGf9-0cd5-n7sBraQ/monthly"]

In [4]:
res = [requests.get(link) for link in links]

In [14]:
# save req to html file
# for i,r in enumerate(res):
#     open("req"+str(i)+".html","w",encoding="UTF-8").write(r.text)

In [15]:
data = [re.findall("[0-9]+-[0-9]+-[0-9]+,[0-9]+",r.text) for r in res]
# data = re.findall("[0-9]+-[0-9]+-[0-9]+,[0-9]+",res[0].text)

In [16]:
data

[['2018-04-09,339',
  '2018-04-10,442',
  '2018-04-11,285',
  '2018-04-12,159',
  '2018-04-13,170',
  '2018-04-14,158',
  '2018-04-15,149',
  '2018-04-17,191',
  '2018-04-18,66',
  '2018-04-19,1239',
  '2018-04-20,46',
  '2018-04-21,252',
  '2018-04-22,810',
  '2018-04-24,120',
  '2018-04-25,1471',
  '2018-04-26,400',
  '2018-04-27,318',
  '2018-04-28,388',
  '2018-04-29,211',
  '2018-04-30,218',
  '2018-05-01,1617',
  '2018-05-02,591',
  '2018-05-03,446',
  '2018-05-04,131',
  '2018-05-05,226',
  '2018-05-06,330',
  '2018-05-07,702',
  '2018-05-08,787',
  '2018-05-09,409',
  '2018-05-10,229',
  '2018-05-11,81',
  '2018-05-12,355',
  '2018-05-13,48',
  '2018-05-14,133',
  '2018-05-15,116',
  '2018-05-16,192',
  '2018-05-17,57',
  '2018-05-18,101',
  '2018-05-19,220',
  '2018-05-20,157',
  '2018-05-21,465',
  '2018-05-22,766',
  '2018-05-23,420',
  '2018-05-24,236',
  '2018-05-25,213',
  '2018-05-26,275',
  '2018-05-27,264',
  '2018-05-28,159',
  '2018-05-29,272',
  '2018-05-30,222',
  

In [20]:
pd.DataFrame(data)

,0
0,"2018-04-09,339"
1,"2018-04-10,442"
2,"2018-04-11,285"
3,"2018-04-12,159"
4,"2018-04-13,170"
...,...
1651,"2018-12-1,13445221"
1652,"2019-01-1,14285260"
1653,"2019-02-1,14096947"
1654,"2019-03-1,15074813"
